## Визуализация торгов за день
- по реальным записям из базы
- тестовые торговые алгоритмы

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'

In [ ]:
from lib.ticker_cache import TickerCache
from dto.config_dto import ConfigDTO
from datetime import datetime
from lib.visaualize import Visualize
from lib.historical_trade import HistoricalTrade
from test_env.test_alg import TestAlgorithm

historical_trade = HistoricalTrade()
ticker_cache = TickerCache(TOKEN, TICKER)

# плавное падение 2024-04-24, 2024-04-25

date = '2024-04-18'
# date = None

alg_name = '-' #'v5_t60'
# alg_name = None #'v5_t60'

viz = Visualize(ticker_cache)

def show_test_alg(date_):
    if date_ is None:
        date_ = datetime.now().strftime('%Y-%m-%d')
    
    test_alg = TestAlgorithm(
        TOKEN,
        do_printing=True,
        config=ConfigDTO(
            ticker=TICKER,
        
            step_max_cnt=6,
            step_base_cnt=3,
            pretest_period=0,

            step_size=1.4,

            step_set_orders_cnt=1,
            step_lots=1,

            stop_up_p=0,
            stop_down_p=0,
            threshold_sell_steps=0,
            threshold_buy_steps=0,
        ),
    )

    result = test_alg.test(
        last_test_date = date_,
        test_days_num = 1, # тут берем всегда только 1 день для этого файла
        shares_count=0,
    )
    
    print()
    print(f"День: {date_}, 'тестовый', Итог: {result['profit']}"
          # f", Записей: {result['op_cnt']}"
          )
    print(result)

    viz.draw(
        date_, 
        test_alg.accounting_helper.get_deals(),
        test_alg.accounting_helper.get_orders(),
        'текущий вариант тестового алгоритма'
    )

results = historical_trade.get_daily_totals(date, alg_name)

for obj in results:
    print(f"День: {obj.date}, {obj.alg_name}, "
          f"{'Итог' if obj.is_closed else 'Подытог'}: {obj.total}, "
          f"Записей: {obj.cnt}")
    viz.draw(
        obj.date, 
        historical_trade.get_deals(obj.date, obj.alg_name, TICKER),
        [],
        obj.alg_name
    )

show_test_alg(date)